## Multi-Site REopt API Script

In [1]:
import pandas as pd
import numpy as np
import openpyxl as xl

import requests
import json
import copy
import time

import csv
from collections import OrderedDict

import os
from src.multi_site_inputs_parser import multi_site_csv_parser
from src.parse_api_responses_to_csv import parse_responses_to_csv_with_template
from src.post_and_poll import get_api_results
from src.post_and_poll import get_run_uuid
from src.parse_api_responses_to_excel import parse_api_responses_to_excel
from src.results_poller import poller

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

%reload_ext autoreload
%autoreload 2

#### To-do: 
#### 1. Code currently uses secondary school ('Sec') load profile for community colleges ('Coll'). Remove relevant lines of code if/when Coll load profiles are included (lines are noted below).

#### Set today's date, choose input site file to run (from Patrick), and align output file name with input file (by date).

In [2]:
#Input today's date.
today_date = '2_24_22'

#Align output file name with Patrick's sites file (label with same date as sites file).
sites_file_name = 'reopt_in_sample_2022-02-09.csv'
output_file_name = 'reopt_output_sample_2022-02-09'

#### Set API key, server, and file paths.

In [3]:
#Set API key and API server. (Yunus to replace with personal API key).
API_KEY = 'n2HFzZ2FlTNJ6ADZsh7X3K4wxiBa4UvdMSgsmkEA'  
server = 'https://developer.nrel.gov/api/reopt/v1'

#Set number of seconds to wait between API post and get request.  
interval = 10

#Set path to input template, which will be filled in with site characteristics from Patrick's input file.   
inputs_path = os.path.join('inputs')
path_to_input_template = os.path.join(inputs_path, 'input_template.csv')

#Set path to outputs folder.
outputs_path = os.path.join('outputs')

#Set path to input site file (from Patrick).
sites_path = os.path.join('inputs/sites')
path_to_sites = os.path.join(sites_path, sites_file_name)

#Set path to utility rate lookup file.
path_to_utility_rate = os.path.join(inputs_path, 'utility_rate_lookup_updated.csv')

#Set path to file needed for battery size estimation.
path_to_battery_size = os.path.join(inputs_path, 'ca_load_profile_cz_mo_20220111.csv')

#Set path to load profiles file.
load_profile_filename = 'LoadProfile_Type_CACZ_DOECZ.csv'
path_to_load_profiles = os.path.join(inputs_path, 'load_profiles/'+load_profile_filename)


### Populate input file with site characteristics needed for REopt run.

#### Set non-site specific parameter values.

In [4]:
parameter_values = {

    #Patrick-derived values
    'ElectricTariff|net_metering_limit_kw' : 1000,
    'ElectricTariff|wholesale_rate_above_site_load_us_dollars_per_kwh' : 0.02,
    'Financial|value_of_lost_load_us_dollars_per_kwh' : 0.58,
    'Generator|diesel_fuel_cost_us_dollars_per_gallon' : 4,
    'LoadProfile|outage_is_major_event' : 'FALSE',
    'Storage|installed_cost_us_dollars_per_kw' : 840,
    'Site|land_acres' : 0,

    #REopt default values (commented out lines are intentionally left blank in input file / are covered by other inputs)
    'ElectricTariff|add_blended_rates_to_urdb_rate' : 'FALSE',
    'ElectricTariff|blended_annual_demand_charges_us_dollars_per_kw' : 0,
    # 'ElectricTariff|blended_annual_rates_us_dollars_per_kwh' :,
    # 'ElectricTariff|blended_monthly_demand_charges_us_dollars_per_kw' :,
    # 'ElectricTariff|blended_monthly_rates_us_dollars_per_kwh' :,
    'ElectricTariff|interconnection_limit_kw' : 100000000,
    # 'ElectricTariff|urdb_rate_name' :,
    # 'ElectricTariff|urdb_response' :,
    'ElectricTariff|wholesale_rate_us_dollars_per_kwh' : 0, 
    'Financial|analysis_years' : 20,
    'Financial|escalation_pct' : 0.026,
    'Financial|microgrid_upgrade_cost_pct' : 0.3,
    'Financial|offtaker_discount_pct' : 0.081,
    'Financial|offtaker_tax_pct' : 0.26,
    'Financial|om_cost_escalation_pct' : 0.025,
    'Generator|existing_kw' : 0,
    'Generator|federal_itc_pct' : 0,
    'Generator|federal_rebate_us_dollars_per_kw' : 0, 
    'Generator|fuel_avail_gal' : 0,
    'Generator|fuel_intercept_gal_per_hr' : 0, 
    'Generator|fuel_slope_gal_per_kwh' : 0,
    'Generator|generator_only_runs_during_grid_outage' : 'TRUE',
    'Generator|generator_sells_energy_back_to_grid' : 'FALSE',
    'Generator|installed_cost_us_dollars_per_kw' : 2500,
    'Generator|macrs_bonus_pct' : 0,
    'Generator|macrs_itc_reduction' : 0, 
    'Generator|macrs_option_years' : 0,
    'Generator|max_kw' : 1000000000,
    'Generator|min_kw' : 0,
    'Generator|min_turn_down_pct' : 0.3,
    'Generator|om_cost_us_dollars_per_kw' : 50,
    'Generator|om_cost_us_dollars_per_kwh' : 20,
    'Generator|pbi_max_us_dollars' : 0,
    'Generator|pbi_system_max_kw' : 0,
    'Generator|pbi_us_dollars_per_kwh' : 0,
    'Generator|pbi_years' : 0,
    'Generator|state_ibi_max_us_dollars' : 0, 
    'Generator|state_ibi_pct' : 0,
    'Generator|state_rebate_max_us_dollars' : 0,
    'Generator|state_rebate_us_dollars_per_kw' : 0,
    'Generator|utility_ibi_max_us_dollars' : 0,
    'Generator|utility_ibi_pct' : 0,
    'Generator|utility_rebate_max_us_dollars' : 0,
    'Generator|utility_rebate_us_dollars_per_kw' : 0,
    #'LoadProfile|critical_loads_kw' :,
    'LoadProfile|critical_loads_kw_is_net' : 'FALSE',
    #'LoadProfile|loads_kw' :,
    'LoadProfile|loads_kw_is_net' : 'TRUE',
    #'LoadProfile|monthly_totals_kwh' :,
    'LoadProfile|year' : 2017,
    'PV|array_type' : 1,
    'PV|azimuth' : 180,
    'PV|dc_ac_ratio' : 1.1,
    'PV|degradation_pct' : 0.005,
    'PV|existing_kw' : 0,
    'PV|federal_itc_pct' : 0.3,
    'PV|federal_rebate_us_dollars_per_kw' : 0,
    'PV|gcr' : 0.4,
    'PV|inv_eff' : 0.96,
    'PV|losses' : 0.14,
    'PV|macrs_bonus_pct' : 0,
    'PV|macrs_itc_reduction' : 0.5,
    'PV|macrs_option_years' : 5,
    'PV|max_kw' : 1000000000,
    'PV|min_kw' : 0,
    'PV|module_type' : 0,
    'PV|om_cost_us_dollars_per_kw' : 16,
    'PV|pbi_max_us_dollars' : 1000000000,
    'PV|pbi_system_max_kw' : 1000000000,
    'PV|pbi_us_dollars_per_kwh' : 0,
    'PV|pbi_years' : 1,
    'PV|radius' : 0,
    'PV|state_ibi_max_us_dollars' : 10000000000,
    'PV|state_ibi_pct' : 0,
    'PV|state_rebate_max_us_dollars' : 10000000000,
    'PV|state_rebate_us_dollars_per_kw' : 0,
    #PV|tilt :,
    'PV|utility_ibi_max_us_dollars' : 10000000000,
    'PV|utility_ibi_pct' : 0,
    'PV|utility_rebate_max_us_dollars' : 10000000000,
    'PV|utility_rebate_us_dollars_per_kw' : 0,
    'Storage|battery_replacement_year' : 10,
    'Storage|canGridCharge' : 'TRUE',
    'Storage|internal_efficiency_pct' : 0.975,
    'Storage|inverter_efficiency_pct' : 0.96,
    'Storage|inverter_replacement_year' : 10,
    'Storage|macrs_bonus_pct' : 0,
    'Storage|macrs_itc_reduction' : 0.5,
    'Storage|macrs_option_years' : 7,
    'Storage|max_kw' : 1000000,
    'Storage|max_kwh' : 1000000,
    'Storage|min_kw' : 0,
    'Storage|min_kwh' : 0,
    'Storage|rectifier_efficiency_pct' : 0.96,
    'Storage|replace_cost_us_dollars_per_kw' : 460,
    'Storage|replace_cost_us_dollars_per_kwh' : 230,
    'Storage|soc_init_pct' : 0.5,
    'Storage|soc_min_pct' : 0.2,
    'Storage|total_itc_pct' : 0,
    'Storage|total_rebate_us_dollars_per_kw' : 0,
    'time_steps_per_hour' : 1,
    'timeout_seconds' : 295,
    #user_uuid :
    
}

#### Use sites file, lookup tables, and load profiles file to populate input template.

In [5]:
#Read in sites file. Change name of utility column.
sites = pd.read_csv(path_to_sites)
sites.rename(columns={'utility':'utility_name_input_file'},inplace=True)

#Read in input template to populate with site characteristics.
input_template = pd.read_csv(path_to_input_template)

#Read in lookup tables for utility rate and battery size.
utility_rate = pd.read_csv(path_to_utility_rate)
utility_rate.rename(columns={'utility_name_gis':'utility_name_input_file'}, inplace=True)

battery_size = pd.read_csv(path_to_battery_size)

#Read in load profiles csv.
load_profiles = pd.read_csv(path_to_load_profiles)

#Columns to pass from sites file to input_template.
cols_to_pass = ['category', 'cz_cec_doe', 'city_ind', 'utility_name_input_file', 'Freq',
       'roof_sqft.x', 'OID', 'latitude', 'longitude', 'roof_sqft.y',
       'floor_sqft', 'diff', 'month', 'n', 'min_io', 'min_irr', 'max_p',
       'max_punmet', 'num_floors', 'in_over_out_mo', 'clp_index', 'clp_name',
       'clp_multiplier', 'critical_load_pct', 'dur_index', 'dur_name',
       'dur_duration', 'roof_sqft']

#Transfer site characteristics from sites file to input template (ex. roof_sqft)
for col in cols_to_pass:
    input_template[col] = sites[col]
    
#Fill in input template columns with non-site-specific parameter values set above.
for col in input_template.columns:
    if col in parameter_values.keys():
        input_template[col] = parameter_values[col]

In [6]:
#### Remove the lines below if/when Community College load profiles are created.
input_template['category_original']=input_template['category']
input_template['category'] = input_template['category'].str.replace('Coll','Sec')

In [7]:
#Building category + climate zone = load profile code for lookup in load profile csv.
input_template['load_profile_code'] = input_template['category'] + '_' + input_template['cz_cec_doe']
input_template['load_file'] = load_profile_filename

#Group input template by load profile code.
grouped = input_template.groupby(['load_profile_code'])

#Loop through load profile groups, getting the annual and peak load per sqft from load profile csv. Rejoin groups into one file.

inputs = pd.DataFrame()
for name, group in grouped:
    
    if name in load_profiles.columns:
        max_kw_per_sqft = max(load_profiles[name])
        group['max_kw_per_sqft'] = max_kw_per_sqft

        annual_kwh_per_sqft = load_profiles[name].sum()
        group['annual_kwh_per_sqft'] = annual_kwh_per_sqft

        inputs = pd.concat([inputs, group],ignore_index=True)
    else:
        #Print load profile code if there's no match in load profile csv.
        print(name)
        
#Scale annual and peak load by floor sqft.
inputs['load_max_kw'] = inputs['max_kw_per_sqft']*inputs['floor_sqft']
inputs['LoadProfile|annual_kwh'] = inputs['annual_kwh_per_sqft']*inputs['floor_sqft']

#### Get URDB label (code for a specific utility rate) for each site based on utility name and building peak load.

In [8]:
#Match utility name from inputs file to utility name in URDB database.
for row in inputs.index:
    utility_name_input_file = inputs.loc[row,'utility_name_input_file']
    utility_name_input_file_inds = utility_rate['utility_name_input_file']== utility_name_input_file 
    utility_name_urdb = utility_rate.loc[utility_name_input_file_inds, 'utility_name_urdb'].iloc[0]
    inputs.loc[row, 'ElectricTariff|urdb_utility_name'] = utility_name_urdb
    
#Get URDB label based on utility name and max power. 
for row in inputs.index:
    max_kw = inputs.loc[row,'load_max_kw']
    utility = inputs.loc[row,'ElectricTariff|urdb_utility_name']
    
    utility_inds = utility_rate['utility_name_urdb'] == utility
    
    min_kw_inds = utility_rate['min_kw']<= max_kw
    
    max_kw_inds = utility_rate['max_kw']>= max_kw 
    nan_kw_inds = utility_rate['max_kw'].isna()
    both_inds = max_kw_inds | nan_kw_inds
    
    filtered_utility_rate = utility_rate.loc[utility_inds & both_inds & min_kw_inds]
    if filtered_utility_rate.empty:
        filtered_utility_rate = utility_rate.loc[utility_inds & min_kw_inds].tail(1)
    
    urdb_label = filtered_utility_rate['urdb_label'].item()

    inputs.loc[row, 'ElectricTariff|urdb_label'] = urdb_label

#### Estimate solar PV size based on roof sqft. Estimate solar cost based on solar PV size estimate.

In [9]:
inputs['Site|roof_squarefeet'] = inputs['roof_sqft']
inputs['pv_kw_estimate'] = inputs['Site|roof_squarefeet']*14/1000/2
inputs['PV|installed_cost_us_dollars_per_kw'] = 4000*(inputs['pv_kw_estimate']**(-0.10))

#### Set critical load pct, outage start times, and outage end times for each site.

In [20]:
inputs['LoadProfile|critical_load_pct']=inputs['critical_load_pct']

#Map month to outage start hour (first Tuesday of every month at 8am).
month_to_outage_start_hour_dict = {
    1:33, 
    2:873, 
    3:1545, 
    4:2217, 
    5:2889, 
    6:3729, 
    7:4401, 
    8:5241, 
    9:5913, 
    10:6585, 
    11:7425, 
    12:8097}

#Determine outage end hour based on outage start hour and outage duration.
inputs['LoadProfile|outage_start_hour']= inputs['month'].apply(lambda x:month_to_outage_start_hour_dict[x])
inputs['LoadProfile|outage_end_hour']= inputs['LoadProfile|outage_start_hour']+inputs['dur_duration']

#### Set description to include key site characteristics.

In [22]:
#Get utility abbreviation to use for site description.
def utility_abbrev(row):
    return [s[0] for s in row['utility_name_input_file'].split()]

abbrev_list = inputs.apply(utility_abbrev, axis=1)
abbrev_list = [''.join(s) for s in abbrev_list]
abbrev_list_col = pd.Series(abbrev_list)

inputs['utility_abbrev'] = abbrev_list_col

#Create site description column.
def create_description(row):
    return str(row['category_original']+'_'+row['cz_cec_doe']+'_'+row['utility_abbrev']+'_'+str(row['city_ind']) +'_c'+ str(row['LoadProfile|critical_load_pct'])+'_d'+str(row['dur_duration'])+'_s'+str(row['LoadProfile|outage_start_hour'])+'_r'+str(row['roof_sqft'])+'_f'+str(row['floor_sqft']))

inputs['description'] = inputs.apply(create_description, axis=1)


#### Estimate battery size based on critical load fraction, outage duration, monthly avg power, and monthly avg irradiance.

In [25]:
#Create column that indiciates if critical load pct is > 0.5.
clp_over_half_inds = inputs['critical_load_pct']>=0.5
clp_under_half_inds = inputs['critical_load_pct']<0.5
inputs.loc[clp_over_half_inds,'clp>0.5']='True'
inputs.loc[clp_under_half_inds,'clp>0.5']='False'

#Group sites by characteristics needed to estimate battery size.
grouped_battery_size = inputs.groupby(['cz_cec_doe','category_original','month','clp>0.5'])

#Loop through site groups, estimate battery size per sqft.

inputs_with_battery_size = pd.DataFrame()

for name, group in grouped_battery_size:
    cz = name[0]
    category = name[1]
    month = name[2]
    clp_over_half = name[3]
    
    cz_inds = battery_size['cz_cec_doe']== cz
    category_inds = battery_size['category']== category
    month_inds = battery_size['month']== month
    
    #Get first of these values in each group, since the values are the same in urban and rural areas (city_ind 0 or 1).
    irr = battery_size[cz_inds & category_inds & month_inds]['Irr'].iloc[0]
    power_avg = battery_size[cz_inds & category_inds & month_inds]['P_avg_per_floor_sqft'].iloc[0]
    
    #Use different equations for battery size if clp >= 0.5 vs <0.5.
    if clp_over_half == 'False':
        group['battery_size_kwh_per_sqft'] = 39.3*(group['critical_load_pct']**2)*power_avg + 0.185 * (group['critical_load_pct']**2) * power_avg * group['dur_duration'] - 3.67*10**(-6) * irr + 0.0251
    elif clp_over_half == 'True':    
        group['battery_size_kwh_per_sqft'] = 0.840 * (group['critical_load_pct']**2) * power_avg * group['dur_duration'] - 11.1*10**(-6) * irr + 0.073

    inputs_with_battery_size = pd.concat([inputs_with_battery_size, group],ignore_index=True)
    
#Multiply battery size (kWh/floor sqft) by floor_sqft to get battery size estimate (kWh).
inputs_with_battery_size['battery_size_kwh_estimate'] = inputs_with_battery_size['battery_size_kwh_per_sqft']*inputs_with_battery_size['floor_sqft']

#### Estimate battery cost based on battery size estimate.

In [75]:
#Estimate battery cost.
inputs_with_battery_size['Storage|installed_cost_us_dollars_per_kwh'] = 1000*np.power(inputs_with_battery_size['battery_size_kwh_estimate'],(-0.19))

#Set minimum battery cost to $120/kwh.
inputs_with_battery_size.loc[inputs_with_battery_size['Storage|installed_cost_us_dollars_per_kwh']<120,'Storage|installed_cost_us_dollars_per_kwh'] = 120
#Set maximum battery cost to $1000/kwh.
inputs_with_battery_size.loc[inputs_with_battery_size['Storage|installed_cost_us_dollars_per_kwh']>1000,'Storage|installed_cost_us_dollars_per_kwh'] = 1000


In [ ]:
#Create site_id based on index of input file.
inputs_with_battery_size['site_id']=inputs_with_battery_size.index

#### Remove line below when coll load profiles are included.

In [ ]:
coll_site_ids = inputs_with_battery_size[inputs_with_battery_size['category_original']=='Coll']['site_id'].tolist()

#### Save input file.

In [33]:
inputs_with_battery_size.to_csv('inputs/api_input_files/input_file_{}.csv'.format(today_date))

### Construct posts for each site and send to REopt API.

In [34]:
#Read in input file saved in last step.
path_to_api_inputs = 'inputs/api_input_files/input_file_{}.csv'.format(today_date)

In [37]:
#Transform contents of input file to list of posts in the correct format for REopt API.
list_of_posts = multi_site_csv_parser(path_to_api_inputs, api_url=server, API_KEY=API_KEY, start=0, n_sites=10)

In [52]:
#Create folder (labeled with today's date) to store output files for each site. 

outputs_folder_name = 'outputs_' + today_date
outputs_folder = os.path.join(outputs_path, outputs_folder_name)

if not os.path.exists(outputs_folder):
    os.makedirs(outputs_folder)

In [56]:
#Loop through list of posts, send posts to REopt API and store results.

#responses = []

break_ = False
for post in list_of_posts:

    #Post to REopt API and get run_uuid associated with that post.
    run_id = get_run_uuid(post, API_KEY=API_KEY, api_url=server)

    #Set name of results file to name of site.
    site_name = post['Scenario']['description']
    print(site_name)
    site_file_name = site_name + '.json'
    results_file = os.path.join(outputs_folder, site_file_name)

    #Check for results from API using run_uuid from recent post. 
    finished = False
    while not finished:
        #Send get request to REopt to get results for the site.
        url_results = server + '/job/{}/results/?api_key={}'.format(run_id , API_KEY)
        get = requests.get(url=url_results, verify=False)
        
        results_output = json.loads(get.content)

        status = results_output['outputs']['Scenario']['status']

        if status != "Optimizing...":
            finished = True
        else:
            time.sleep(interval)
    
    input_file = pd.read_csv(path_to_api_inputs)
    
    #Pass through the following site characteristics from the input sites file to the output file for each site. 
    
    clp_multiplier = input_file.loc[input_file['description']==site_name]['clp_multiplier'].item()
    results_output['inputs']['Scenario']['Site']['clp_multiplier']= clp_multiplier
    
    min_io = input_file.loc[input_file['description']==site_name]['min_io'].item()
    results_output['inputs']['Scenario']['Site']['min_io']= min_io
    
    min_irr = input_file.loc[input_file['description']==site_name]['min_irr'].item()
    results_output['inputs']['Scenario']['Site']['min_irr']= min_irr
    
    max_p = input_file.loc[input_file['description']==site_name]['max_p'].item()
    results_output['inputs']['Scenario']['Site']['max_p']= max_p
    
    max_p_unmet = input_file.loc[input_file['description']==site_name]['max_p_unmet'].item()
    results_output['inputs']['Scenario']['Site']['max_p_unmet']= max_p_unmet
    
    month = input_file.loc[input_file['description']==site_name]['month'].item()
    results_output['inputs']['Scenario']['Site']['month']= float(month)
    
    floor_sqft = input_file.loc[input_file['description']==site_name]['floor_sqft'].item()
    results_output['inputs']['Scenario']['Site']['floor_sqft']= float(floor_sqft)
    
    freq = input_file.loc[input_file['description']==site_name]['Freq'].item()
    results_output['inputs']['Scenario']['Site']['freq']= float(freq)
    
    urban_rural = input_file.loc[input_file['description']==site_name]['city_ind'].item()
    results_output['inputs']['Scenario']['Site']['city_ind']= float(urban_rural)
    
    category = input_file.loc[input_file['description']==site_name]['category_original'].item()
    results_output['inputs']['Scenario']['Site']['category']= category
    
    utility = input_file.loc[input_file['description']==site_name]['utility_name_input_file'].item()
    results_output['inputs']['Scenario']['Site']['utility']= utility
    
    cz = input_file.loc[input_file['description']==site_name]['cz_cec_doe'].item()
    results_output['inputs']['Scenario']['Site']['cz_cec_doe']= cz

    site_id = input_file.loc[input_file['description']==site_name]['site_id'].item()
    results_output['inputs']['Scenario']['site_id']= site_id
    
    crit_load_pct = input_file.loc[input_file['description']==site_name]['critical_load_pct'].item()
    results_output['inputs']['Scenario']['Site']['critical_load_pct']= crit_load_pct
    
    outage_dur = input_file.loc[input_file['description']==site_name]['dur_duration'].item()
    results_output['inputs']['Scenario']['Site']['outage_duration']= outage_dur
    
    outage_start = input_file.loc[input_file['description']==site_name]['LoadProfile|outage_start_hour'].item()
    results_output['inputs']['Scenario']['Site']['outage_start_hour']= outage_start
    

#Uncomment section below to run outage simulation (outage starting in every hour of the year) for each site:

#     #Send post request to run outage simulation for site.
#     post_text = {"run_uuid"= run_id, "bau"= True}
#     url_outagesim = server + '/outagesimjob?API_KEY=' + API_KEY
#     outagesim_post = requests.post(url_outagesim, json-post_text)

#     #Send get request to resilience_stats endpoint, sleep for time interval if results aren't ready.
#     finished = False
#     while not finished:
#         #Send get request for resilience stats for the site.
#         url_resilience = server + '/job/{}/resilience_stats?API_KEY={}&bau=true'.format(run_id, API_KEY)
#         resilience_output = json.loads(requests.get(url_resilience).content)

#         for key in resilience_output:
#             if key in ['Error']:
#                 if resilience_output['Error'] == 'Outage sim results are not ready. If you have already submitted an outagesimjob, please try again later. If not, please first submit an outagesimjob by sending a POST request to v1/outagesimjob/ with run_uuid and bau parameters. This will generate outage simulation results that you can access from a GET request to the v1/job/<run uuid>/resilience_stats endpoint. Sample body data for POST-ing to /outagesimjob/= {"run_uuid"= "6ea30f0f-3723-4fd1-8a3f-bebf8a3e4dbf", "bau"= false}':  
#                     time.sleep(interval)
#                 else:
#                     print('run_id=' + run_id)
#                     print('count=' + str(count))
#                     print(resilience_output)
#                     finished = True
#                     break_ = True
#             else:
#                 finished = True

#     if break_ == True:
#         break

#     #Appends resilience output dictionary to results output dictionary.
#     resilience_output_trim = dict(ele for sub in resilience_output.values() for ele in sub.items())
#     results_output['outputs']['Scenario']['Site']['Resilience']=resilience_output_trim


    #Write REopt results for the given site in the corresponding site output file.
    with open(results_file, 'w') as fp:
        json.dump(obj=results_output, fp=fp)

    #Append site results to responses list to write all site results to Excel sheet in next step.
    #responses.append(results_output)


main         INFO     Response OK from https://developer.nrel.gov/api/reopt/v1/job/?api_key=n2HFzZ2FlTNJ6ADZsh7X3K4wxiBa4UvdMSgsmkEA.


CC_10_3B_SDG&E_1_c0.48742137_d96_s1545_r35200_f42200


main         INFO     Response OK from https://developer.nrel.gov/api/reopt/v1/job/?api_key=n2HFzZ2FlTNJ6ADZsh7X3K4wxiBa4UvdMSgsmkEA.


CC_10_3B_SCE_1_c0.48742137_d96_s1545_r6000_f7200


main         INFO     Response OK from https://developer.nrel.gov/api/reopt/v1/job/?api_key=n2HFzZ2FlTNJ6ADZsh7X3K4wxiBa4UvdMSgsmkEA.


...

CC_10_3B_SDG&E_0_c0.9358490309999999_d48_s1545_r1200_f1200


main         INFO     Response OK from https://developer.nrel.gov/api/reopt/v1/job/?api_key=n2HFzZ2FlTNJ6ADZsh7X3K4wxiBa4UvdMSgsmkEA.


CC_10_3B_SDG&E_0_c1.559748385_d192_s1545_r11700_f11700


main         INFO     Response OK from https://developer.nrel.gov/api/reopt/v1/job/?api_key=n2HFzZ2FlTNJ6ADZsh7X3K4wxiBa4UvdMSgsmkEA.


CC_10_3B_SCE_0_c0.584905644_d192_s1545_r118800_f118800


### Write results to Excel file.

#### Loop through output files for each site (json files) and append contents to a list.

In [57]:
directory = 'outputs' + '/outputs_' + today_date
responses = []

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith('.json'):
        filepath = 'outputs/' +'outputs_' + today_date + '/'+ str(filename)
        f = open(filepath,'r')
        contents = json.loads(f.read())
        responses.append(contents)

#### Writes contents of list to Excel file.

In [58]:
parse_api_responses_to_excel(responses, spreadsheet='{}.xlsx'.format(output_file_name))

/opt/anaconda3/lib/python3.8/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
